In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

openai_api_key= os.getenv("OPENAI_API_KEY")

from langchain_openai import ChatOpenAI

llm= ChatOpenAI(model="gpt-4.1-2025-04-14")
llm.invoke("hi")

d:\IBM_Agentic_AI\prompt_Day_1\venv12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AIMessage(content='Hello! How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_1a2c4a5ede', 'id': 'chatcmpl-CopARVIIyDQgydHTi6p3eBMBsLsdA', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b3b75-a15e-7682-b817-1cbea3894d91-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Preprocessing

Load the document

In [2]:
import wget
file_name='companypolicies.txt'
url="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/6JDbUb_L3egv_eOkouY71A.txt"

# use wget to download the file
wget.download(url,out=file_name)
print('file download')

file download


In [3]:
with open(file_name,'r') as file:
    content=file.read()
    print(content)

1.	Code of Conduct

Our Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization. We are committed to maintaining a workplace that is built on integrity, respect, and accountability.
Integrity: We hold ourselves to the highest ethical standards. This means acting honestly and transparently in all our interactions, whether with colleagues, clients, or the broader community. We respect and protect sensitive information, and we avoid conflicts of interest.
Respect: We embrace diversity and value each individual's contributions. Discrimination, harassment, or any form of disrespectful behavior is unacceptable. We create an inclusive environment where differences are celebrated and everyone is treated with dignity and courtesy.
Accountability: We take responsibility for our actions and decisions. We follow all relevant laws and regulations, and we strive to continuously improve our practices. We report any potential violations of 

Splitting the document into chunks

In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

In [5]:
loader =TextLoader(file_name)
document = loader.load()
text_splitter =CharacterTextSplitter(chunk_size=1200,chunk_overlap=200)
texts=text_splitter.split_documents(document)
print(len(texts))

Created a chunk of size 1624, which is longer than the specified 1200
Created a chunk of size 1885, which is longer than the specified 1200
Created a chunk of size 1903, which is longer than the specified 1200
Created a chunk of size 1729, which is longer than the specified 1200
Created a chunk of size 1678, which is longer than the specified 1200
Created a chunk of size 2032, which is longer than the specified 1200
Created a chunk of size 1894, which is longer than the specified 1200


16


Embeeding and storing

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

In [7]:
embeddings=HuggingFaceEmbeddings()
docsearch=Chroma.from_documents(documents=texts,
                                embedding=embeddings)
print("document ingested")

document ingested


Integrated Lnagchain

In [8]:
from langchain_classic.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate

In [9]:
model =ChatOpenAI(model="gpt-5-2025-08-07")

In [10]:
qa=RetrievalQA.from_chain_type(llm=model,
                               chain_type='stuff',
                               retriever=docsearch.as_retriever(),
                               return_source_documents=False)
query="what is mobile ploicy?"
response=qa.invoke(query)

print(response)

{'query': 'what is mobile ploicy?', 'result': 'If you’re asking about our Mobile Phone Policy, it sets the standards for appropriate, secure, and compliant use of mobile devices at work. Key points:\n\n- Acceptable use: Phones are primarily for work; limited personal use is fine if it doesn’t disrupt work.\n- Security: Protect your device and credentials; be cautious with apps/links; report any security concerns promptly.\n- Confidentiality: Don’t send sensitive company info via unsecured apps or email; be discreet in public.\n- Cost management: Keep personal use separate from company accounts; reimburse any personal charges on company-issued phones.\n- Compliance: Follow all applicable data protection and privacy laws.\n- Lost/stolen: Report immediately to IT or your supervisor.\n- Consequences: Non-compliance may lead to discipline, including loss of phone privileges.\n\nAll employees are expected to understand and follow this policy, which is reviewed regularly to keep up with techn

In [11]:
qa=RetrievalQA.from_chain_type(llm=model,
                               chain_type='stuff',
                               retriever=docsearch.as_retriever(),
                               return_source_documents=False)
query="Can you summarize the document for me?"
response=qa.invoke(query)

print(response)

{'query': 'Can you summarize the document for me?', 'result': 'Here’s a concise summary of the provided document:\n\n- Purpose: Establishes the organization’s ethical foundation and expectations for all employees.\n\n- Core Code of Conduct principles:\n  - Integrity: Act honestly and transparently; protect sensitive information; avoid conflicts of interest.\n  - Respect: Embrace diversity; prohibit discrimination and harassment; foster an inclusive, courteous workplace.\n  - Accountability: Take responsibility; comply with laws and regulations; continuously improve; report potential violations and support investigations.\n  - Safety: Prioritize safety and report unsafe conditions or practices.\n  - Environmental Responsibility: Minimize environmental impact and promote sustainability.\n\n- Expectations: All employees must uphold these standards and serve as role models to protect the organization’s reputation for ethical conduct and social responsibility.\n\n- Health and Safety Policy:

Using PROMPT Template

In [12]:
from langchain_core.prompts import PromptTemplate

In [13]:
prompt_template = """Use the information from the document to answer the question at the end. If you don't know the answer, just say that you don't know, definately do not try to make up an answer.

{context}

Question: {question}
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

In [14]:
qa=RetrievalQA.from_chain_type(llm=model,
                               chain_type='stuff',
                               retriever=docsearch.as_retriever(),
                               chain_type_kwargs=chain_type_kwargs,
                               return_source_documents=False)
query="Can I eat in company vehicles?"
response=qa.invoke(query)

print(response)

{'query': 'Can I eat in company vehicles?', 'result': 'I don’t know. The provided policies only state that smoking is not permitted in company vehicles; they don’t mention eating. Please check the vehicle use policy or ask HR/your manager for guidance.'}


Make the conversation have memory

Create a history list to store the chat history

In [15]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough


In [16]:
chat_history = ChatMessageHistory()

In [36]:
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "Use the information from the document to answer the question at the end. "
     "If you don't know the answer, just say that you don't know, definately do not try to make up an answer."),
    
    MessagesPlaceholder(variable_name="chat_history"),
    
    ("human", "{question}")
])

In [37]:
retriever = docsearch.as_retriever()

In [38]:
rag_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "chat_history": lambda _: chat_history.messages
    }
    | prompt
    |model
)


In [39]:
def qa(question: str | None = None):
    if question is None:
        question = input("Question: ")

    response = rag_chain.invoke(question)

    chat_history.add_user_message(question)
    chat_history.add_ai_message(response.content)

    return response.content


In [40]:
qa("what is mobile policy")

'I don’t know—no document or context was provided. Which “mobile policy” do you mean (e.g., your company’s mobile device/BYOD policy, a carrier plan policy, or an app store policy)? If you share the document or details, I can answer precisely.'